In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os, sys
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str('../../../../../aws_google/qfef-f18ffa4ac76b.json')

sys.path.append('../')
from utils.google_nlp import *

In [3]:
#database connection settings
import psycopg2

db_name = "traviato_development"
db_host = "localhost"
db_port = "5432"
db_user = "lievgarcia"
db_pwd = "traviato81"

conn = psycopg2.connect(database=db_name, user=db_user, password=db_pwd, host=db_host, port=db_port)

In [12]:
import pandas as pd

cursor = conn.cursor()  
query = "SELECT id, source_id, title, work, composer, theatre, review_date, full_text, rating " + \
        "FROM reviews l " + \
        "WHERE sentiment_score IS NULL and full_text != 'nan' " + \
        "order by review_date "
        
cursor.execute(query)
columns = ['id', 'source_id', 'title', 'work', 'composer', 'theatre', 'review_date', 'full_text', 'rating']

r_df = pd.DataFrame(cursor.fetchall(), columns=columns)
r_df.shape

(0, 9)

In [13]:
for ix, row in r_df.iterrows():
    score, magnitude, sentences = sentiment_text(row['full_text'])
    cursor = conn.cursor()  
    query = "UPDATE reviews SET " + \
            "sentiment_score = {}".format(score) + ', sentiment_magnitude = {}'.format(magnitude) + " " + \
            "where id = {}; ".format(row['id'])
    commit = 'COMMIT;'
    
#     print(query + commit)
    cursor.execute(query + commit)